In [ ]:
pip install -q yahoo_fin

In [2]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml@v0.0.6#egg=runml

In [4]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [5]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HIMS','HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-6a', mod, True, loss=lossfn)[0]

In [ ]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
later = [ ]
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-5b', mod, False)


In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
10,IPO,RROnly,24.529832,0.962766,251.14,320.67,571.81,39.56,43.340000,0.10,9.255319
1,ARKK,RROnly,31.119823,0.920213,579.44,582.25,1161.69,43.32,47.980000,0.11,8.476789
2,ARKW,RROnly,38.526062,0.898936,674.75,617.32,1292.07,76.67,89.940002,0.17,8.394869
44,XME,RROnly,20.575078,0.914894,307.54,177.60,485.14,59.06,54.880001,-0.07,7.933131
33,XLB,RROnly,53.212842,0.936170,354.16,224.51,578.67,89.64,85.639999,-0.04,7.765957
32,XITK,RROnly,92.808587,0.898936,775.39,1097.26,1872.65,142.49,153.740005,0.08,7.726064
0,ARKF,RROnly,14.373508,0.856383,177.71,218.92,396.63,27.58,32.310001,0.17,7.719024
21,ONLN,RROnly,27.056857,0.888298,235.93,327.51,563.44,40.31,43.939999,0.09,7.641844
31,WDIV,RROnly,44.056150,0.941489,162.92,113.67,276.60,59.91,58.029999,-0.03,7.464539
15,IWO,RROnly,193.895751,0.936170,1032.89,1069.89,2102.77,260.22,267.140015,0.03,7.234043


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers3 = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CLOV', 'CRCT', 'CXM', 'CZOO',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MRDB', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-6b', mod, True)[0]

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
7,AUGX,RROnly,0.900917,0.920000,29.16,46.83,75.99,0.80,1.250000,0.55,5.266310
6,APPS,RROnly,2.952001,0.920213,421.94,498.77,920.71,1.46,2.060000,0.41,5.218413
19,CXM,RROnly,7.950363,0.942029,62.87,110.06,172.93,8.73,10.210000,0.17,5.200341
49,ONDS,RROnly,0.530255,0.903614,45.13,66.79,111.92,0.69,0.910000,0.32,5.014174
23,DMTK,RROnly,1.040155,0.835106,238.36,234.68,473.04,0.11,0.540000,3.70,4.867825
61,RVYL,RROnly,3.430919,0.856383,389.76,908.20,1297.96,1.46,2.430000,0.67,4.823193
47,NNOX,RROnly,6.048989,0.861111,253.33,343.85,597.18,6.98,9.920000,0.42,4.734672
70,STEM,RROnly,1.874044,0.873563,178.44,293.66,472.10,1.13,1.470000,0.30,4.717151
68,SOUN,RROnly,1.145403,0.906250,27.92,34.40,62.32,4.00,4.590000,0.15,4.705882
34,INDI,RROnly,5.036675,0.904255,89.63,87.46,177.09,6.38,7.280000,0.14,4.635258


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-5b', mod4, False)[0]

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
2,ADBE,RROnly,282.879698,0.930851,3143.79,3120.10,6263.89,555.54,508.760010,-0.08,5.784574
11,CRM,RROnly,132.181681,0.904255,1213.46,1557.54,2771.00,257.10,225.580002,-0.12,5.661094
34,SBUX,RROnly,67.863903,0.893617,321.18,412.33,733.52,77.85,84.989998,0.09,5.200946
36,TSLA,RROnly,104.519260,0.856383,2280.15,2631.22,4911.37,197.88,176.429993,-0.11,4.811412
37,TXN,RROnly,117.817157,0.888298,740.12,571.89,1312.01,194.53,181.199997,-0.07,4.749240
28,NKE,RROnly,80.874272,0.781915,490.89,566.75,1057.65,75.37,92.989998,0.23,4.636910
21,INTC,RROnly,25.187652,0.835106,213.70,247.25,460.95,30.97,33.480000,0.08,3.903875
22,IBM,RROnly,87.926482,0.861702,521.59,341.11,862.71,172.95,162.580002,-0.06,3.850051
19,FDX,RROnly,145.108907,0.813830,936.71,864.73,1801.43,299.84,274.109985,-0.09,3.744512
30,NOW,RROnly,355.692425,0.824468,3122.51,2940.93,6063.44,786.67,725.390015,-0.08,3.694909


In [ ]:
findata.EPOCHS=200
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-6b', mod5, True)[0]


In [ ]:
addAlloc(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
24,FQVLF,RROnly,7.686009,0.920213,152.29,259.34,411.63,13.20,16.469999,0.25,5.934650
62,TKAMY,RROnly,4.158087,0.930851,73.63,70.06,143.69,4.38,5.060000,0.16,5.784574
2,ADYEY,RROnly,7.129250,0.909574,150.22,166.93,317.16,11.87,14.540000,0.22,5.674910
55,SE,RROnly,40.423803,0.893617,1150.49,1558.02,2708.51,71.42,84.120003,0.18,5.200946
45,OTGLY,RROnly,4.353191,0.819149,55.78,93.30,149.07,8.60,10.740000,0.25,4.404255
3,BAK,RROnly,6.556585,0.909574,137.75,125.20,262.96,6.45,6.900000,0.07,3.913374
14,EADSF,RROnly,68.409674,0.898936,753.03,429.35,1182.38,137.01,147.839996,0.08,3.894377
48,PROSF,RROnly,28.452490,0.845745,370.11,527.37,897.48,37.15,41.970001,0.13,3.667875
57,STLA,RROnly,6.901286,0.909574,116.19,69.37,185.56,19.85,20.969999,0.06,3.482776
61,THQQF,RROnly,1.919248,0.739362,88.38,89.16,177.55,2.22,2.760000,0.24,3.109169


In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'FLN', 'FM',
           'ECH', 'EPHE', 'EWA', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-7a', mod6, True)[0]


In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
23,XCEM,RROnly,22.234866,0.962766,110.18,75.09,185.27,32.75,31.260000,-0.05,8.882979
17,ILF,RROnly,15.852192,0.930851,96.95,89.50,186.44,25.39,23.270000,-0.08,8.444149
15,EWZ,RROnly,20.170528,0.920213,157.05,125.89,282.94,28.39,25.600000,-0.10,8.404255
9,EWC,RROnly,24.565045,0.941489,128.69,80.29,208.98,37.62,35.779999,-0.05,8.244681
20,THD,RROnly,54.419120,0.936170,230.85,225.86,456.71,55.32,57.869999,0.05,8.085106
0,ARGT,RROnly,21.863883,0.893617,228.66,150.28,378.94,56.39,49.360001,-0.12,8.049645
13,EWW,RROnly,29.090125,0.882979,200.34,167.41,367.74,57.28,49.029999,-0.14,7.993921
12,EWT,RROnly,31.234260,0.898936,199.47,101.40,300.87,55.48,50.160000,-0.10,7.978723
8,EWJ,RROnly,47.531534,0.920213,168.58,162.01,330.59,69.96,65.820000,-0.06,7.872340
14,EWY,RROnly,46.431693,0.898936,263.87,217.14,481.02,68.96,63.730000,-0.08,7.726064


In [ ]:
findata.EPOCHS=200
review = ['AMR', 'IFF', 'BLK']
tickers7 = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-7a', mod7, True)[0]

In [ ]:
addAlloc(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
15,CC,RROnly,14.604981,0.946809,234.31,217.03,451.34,22.11,24.040001,0.09,8.877069
13,BXC,RROnly,11.515895,0.925532,831.48,659.30,1490.78,88.69,103.019997,0.16,8.789894
52,KHC,RROnly,24.830637,0.946809,145.15,120.29,265.44,32.09,34.270000,0.07,8.708207
54,KVUE,RROnly,17.823277,0.978261,16.96,24.91,41.87,18.31,18.690001,0.02,8.695652
59,NTR,RROnly,29.697861,0.930851,344.70,395.54,740.23,49.93,54.270000,0.09,8.540189
88,TWI,RROnly,1.816977,0.898936,93.38,99.46,192.84,6.95,8.120000,0.17,8.394869
72,RH,RROnly,221.017628,0.925532,2965.78,3221.09,6186.87,238.02,254.679993,0.07,8.191489
67,PNR,RROnly,38.925294,0.930851,322.62,267.62,590.24,74.39,78.580002,0.06,8.156028
29,DAL,RROnly,25.346671,0.936170,355.07,237.61,592.68,46.02,48.310001,0.05,8.085106
0,AAL,RROnly,11.235896,0.930851,125.11,128.97,254.08,11.04,11.430000,0.04,7.579787


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'FXE', 'FXY', 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-6a', mod8, False)[0]

In [8]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
8,TLT,RROnly,81.800579,0.861702,185.62,392.61,578.22,93.94,88.949997,-0.05,5.851064
6,SOYB,RROnly,14.247738,0.856383,79.20,60.78,139.98,22.99,24.139999,0.05,5.691489
7,TIP,RROnly,99.998441,0.914894,110.06,128.09,238.15,107.51,105.150002,-0.02,4.893617
3,ISHG,RROnly,63.476836,0.872340,56.40,122.52,178.92,71.03,68.989998,-0.03,4.468085
1,FXY,RROnly,58.088600,0.781915,44.38,129.87,174.25,58.62,62.000000,0.06,4.184397
2,GLD,RROnly,153.838371,0.808511,461.11,285.71,746.82,223.11,216.770004,-0.03,1.702128
4,PDBC,RROnly,7.505231,0.803191,48.04,26.13,74.17,14.05,14.440000,0.03,1.471631
0,FXE,RROnly,87.764250,0.595745,42.82,44.19,87.01,100.67,97.370003,-0.03,0.000000
5,SLV,RROnly,17.372754,0.744681,58.20,46.27,104.47,28.12,27.500000,-0.02,0.000000
9,USO,RROnly,30.128333,0.547872,129.53,-36.86,92.67,80.25,79.410004,-0.01,0.000000


In [ ]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-7a', mod9, True)[0]

In [10]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
9,LINK-USD,RROnly,5.661895,0.914894,138.59,143.39,281.97,13.09,15.180000,0.16,6.003040
4,DOGE-USD,RROnly,0.059106,0.845745,1.03,1.35,2.39,0.11,0.160000,0.41,5.664801
15,TRX-USD,RROnly,0.052291,0.882979,0.60,0.42,1.03,0.14,0.120000,-0.17,5.618630
16,XRP-USD,RROnly,0.329530,0.867021,3.93,5.37,9.30,0.54,0.460000,-0.15,5.306484
14,TON-USD,RROnly,1.148060,0.813830,14.08,31.20,45.28,1.17,1.590000,0.36,5.295930
6,ETH-USD,RROnly,1082.072620,0.840426,18305.42,24242.18,42547.60,3156.44,3684.850098,0.17,5.064366
13,SOL-USD,RROnly,13.901543,0.813830,884.38,1215.71,2100.09,140.60,173.960007,0.24,5.037361
11,MATIC-USD,RROnly,0.401464,0.872340,10.69,13.69,24.38,0.52,0.480000,-0.08,4.635258
7,FIL-USD,RROnly,3.527541,0.787234,60.51,189.36,249.87,4.25,4.900000,0.15,4.204661
0,ADA-USD,RROnly,0.274663,0.787234,3.82,8.05,11.86,0.44,0.380000,-0.13,3.986439


In [11]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
cry-7a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,222,139 bytes  received 35 bytes  6,444,348.00 bytes/sec
total size is 221,571,384  speedup is 68.76
/content/drive/MyDrive/colab/results/com-4a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/com-4a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/com-4a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/com-4a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
